In [1]:
# importing all the required modules
import PyPDF2
import re
import random

# creating a pdf reader object
reader = PyPDF2.PdfReader("./Course_Schedule_2023-24-2.pdf")

# print the number of pages in pdf file
print(len(reader.pages))

entries = []
for page in reader.pages:
    entries.append(page.extract_text())

83


In [2]:
print(entries[5].replace("\n", ""))

 Course Schedule/ Classroom Allocation: 2023 -24/2 Semester                                                  [*This is a dynamic list. It will get updated as and when departments propose changes.]   Page 6 of 83 19-Jan-24 5:04:35 PM  S.No. Branch Course Name/Group Name  Slot Name Units(Credits) Course Type  Instructor  Instructor Email  Discussion Schedule  Tutorial Schedule  Practical Schedule singhappu@iitk.ac.in (O)  (BSBE SR) 15:00 -16:00  42 BSBE  BIOINFORMATICS & COMPUTATIONAL BIOLOGY(BSE322)  SLOT -4-4 3-0-0-0(9) DC / REGULAR  NITIN GUPTA(I)  guptan@iitk.ac.in (I) M (L01) Th (L01) 12:00 -13:15    43 BSBE  BIOINFORMATICS & COMPUTATIONAL BIOLOGY(BSE322A)  SLOT -4-4 3-0-0-1(10)  PRF / REGULAR  NITIN GUPTA(I)  guptan@iitk.ac.in (I) M (L01) Th (L01) 12:00 -13:15    44 BSBE  UG PROJECT (UGP -II)(BSE399A)  SLOT - Blank  0-0-2-0(9) UGP -2 / REGULAR  DUGCBSBE (I)  dugc_bsbe@iitk.ac.in (I)     45 BSBE  TISSUE ENGINEERING(BSE421A)  OE-1 3-0-0-2(11)  DE,PRF / REGULAR  DHIRENDRA S KATTI(I)  

In [3]:
example_entry: str = entries[32]
extra_match = re.compile("Practical Schedule(.+?)\s(\d+)\s[A-Z]")
extra_match.search(example_entry.replace("\n", "")).groups()

(' dugc_des@iitk.ac.in (O) ', '284')

In [4]:
number_match = re.compile(r'\n(\d+\s[A-Z].+?(?=\n\d+\s[A-Z]|\Z))', re.DOTALL)

In [5]:
courses_raw = []
for entry in entries:
    courses_raw.extend(number_match.findall(entry))

In [8]:
def remove_nested_parens(input_str):
    result = ''
    paren_level = 0
    for ch in input_str:
        if ch == '(':
            paren_level += 1
        elif (ch == ')') and paren_level:
            paren_level -= 1
        elif not paren_level:
            result += ch
    return result

In [9]:
problematic = [577, 475, 336, 349, 348, 676, 138]

In [98]:
idx = random.randint(1, 700)
course = courses_raw[186]
# r"iitk.ac.in[,\s]*\([IO]\)[,\s]+" <-- takes care of anomaly but at cost
print(course.replace("\n", ""))
rest_time_split = re.split(r"iitk.ac.in\s*\([IO]\),*\s+(?=[A-Z])", course.replace("\n", ""), re.DOTALL)
rest_time_split

187 CHM  GENERAL CHEMISTRY: INORGANIC & ORGANIC CHEMISTRY(CHM113M)/C  CHM112 (1)  2-1-0-0(4) IC / SECOND -HALF  Prakash Mondal(I), Ramkrishna Sarkar(O)  pcmondal@iitk.ac.in (I), ramkrishna@iitk.ac.in (O)  MW 17:00 -18:00  F 15:00 -16:00   


['187 CHM  GENERAL CHEMISTRY: INORGANIC & ORGANIC CHEMISTRY(CHM113M)/C  CHM112 (1)  2-1-0-0(4) IC / SECOND -HALF  Prakash Mondal(I), Ramkrishna Sarkar(O)  pcmondal@iitk.ac.in (I), ramkrishna@',
 'MW 17:00 -18:00  F 15:00 -16:00   ']

In [99]:
course, timing = rest_time_split[0], rest_time_split[1].strip()
course += "iitk.ac.in"

In [100]:
print(timing)
timing_slots = re.findall(r"(.+?)(\d\d:\d\d)\s*-\s*(\d\d:\d\d)", timing)
timing_slots

MW 17:00 -18:00  F 15:00 -16:00


[('MW ', '17:00', '18:00'), ('  F ', '15:00', '16:00')]

In [101]:
for (t, s, e) in timing_slots:
    t = remove_nested_parens(t)
    print(re.findall(r"(Th|M|T|W|F)", t), s, e)

['M', 'W'] 17:00 18:00
['F'] 15:00 16:00


In [102]:
course

'187 CHM  GENERAL CHEMISTRY: INORGANIC & ORGANIC CHEMISTRY(CHM113M)/C  CHM112 (1)  2-1-0-0(4) IC / SECOND -HALF  Prakash Mondal(I), Ramkrishna Sarkar(O)  pcmondal@iitk.ac.in (I), ramkrishna@iitk.ac.in'

In [103]:
course_instructor_split = re.split(r"\d+-\d+-\d+-\d+", course)

In [107]:
course_desc, instructor_desc = course_instructor_split[0], course_instructor_split[1]
course_desc

'187 CHM  GENERAL CHEMISTRY: INORGANIC & ORGANIC CHEMISTRY(CHM113M)/C  CHM112 (1)  '

In [111]:
re.search(r"^(\d+)\s+([A-Z]+)\s+(.*?)\(([A-Z]+\d+[A-Z]*)\)", course_desc).groups()

('187', 'CHM', 'GENERAL CHEMISTRY: INORGANIC & ORGANIC CHEMISTRY', 'CHM113M')

In [83]:
instr_split = re.split(r"\s\s+", instructor_desc)
instr_split

['(5) DC / FIRST -HALF', 'ABHISHEK (I)', 'abhish@iitk.ac.in']

In [84]:
re.search(r"\((\d+)\)", instr_split[0]).groups()[0]

'5'

In [85]:
re.split(r",\s+", instr_split[1])

['ABHISHEK (I)']

In [86]:
instr_split[-3] = remove_nested_parens(instr_split[-3])
re.findall(r"([A-Za-z0-9]+)", instr_split[0])

['DC', 'FIRST', 'HALF']

In [90]:
re.findall(r"([A-Za-z0-9]+)\s+-([A-Za-z0-9]+)", instr_split[0])

[('FIRST', 'HALF')]